# Implementação com base no paper Falkenauer
_____________________________________
#      Primeira População
- Inicializa a primeira população usando First Fit Heuristic 

_______________________________________________________________
#      Crossover (BPCX)

- O funcionamento do Crossover funciona da seguinte maneira:

- Primeiramente os itens serão representados em objetos, na seguinte forma:
*Exemplo:*
    
    - objeto[1] = {id : 1, peso : 5}

    - objeto[2] = {id : 2, peso : 3}
    
    - ...
    
    - objeto[k] = {id : k, peso : x}

    

- Cada letra do alfabeto(independente se for maiuscula ou minuscula) representa uma caixa com uma configuração
aleatória de itens dentro dela.
*Exemplo de Cromossomo:*

    - [chromossomo-K]
    
    - (gene 0)  A = [1, 5, 3]
    
    - (gene 1)  B = [2, 4]           
    
    - (gene 2)  C = [6, 7, 8]
    
    - (gene 3)  D = [9, 10] 



- Seja ABCDEF(o primeiro pai) x abcd(segundo pai)     {perceba que o numero de genes pode ser diferente}

### 1. faz uma copia de ambos os pais

### 2. faz o split aleatorio em pedaços dos pais
*Exemplo*
    
    - A | BCD | EF  
   
    - ab | cd | 

### 3. une ambos de forma arbitraria. Por exemplo:

    AcdBCDEF

### 4. ao unir podem existir itens repetidos, para solucionar isso:
    - remove os bins que possuem geraram ids repetidos nos cromossomos

    - nestes bins deletados exitem itens que ja existiam no cromossomo e itens que nao existiam nele
    
    - ignora os itens repetidos e reinsere os itens que nao existiam no cromossomo usando a heuristica FF

_______________________________________________________________
#      Mutação

*Exemplo*
### 1. Seleciona bins aleatorios e remove eles

    - [chromossome-K]
    
    - (gene 0)  A = [1, 5, 3]
    
    - (gene 1)  B = [2, 4]            (erase bin B)
    
    - (gene 2)  C = [6, 7, 8]
    
    - (gene 3)  D = [9, 10]           (erase bin D)


### 2. reinsere os itens 2, 4, 9, 10 usando FFD

### 3. Deve seguir as seguintes regras:
    
    - Dentre os bins deletados 
        - Um deve ser o mais vazio
        - os outros devem ser randomicos
        - Devem ser deletados no minimo 3 bins

_______________________________________________________________
#      Inversão

### 1. swap dois bins aleatorios de um cromossomo
    - esse swap pode ser promissor pois aumenta a probabilidade de que dois bins praticamente cheios podem
    diminuir a distancia entre si, fazendo com que essa caracteristica possa passar para as proximas gerações

In [76]:
import random
import math
import matplotlib.pyplot as plt

In [77]:
def read_input_from_file(file_path):
  with open(file_path, 'r') as file:
    # Ler a primeira linha que contém N
    N = int(file.readline().strip())  # Quantidade de itens
    
    # Ler a segunda linha que contém C
    C = int(file.readline().strip())  # Capacidade de cada caixa
    
    # Ler os pesos dos itens, um por linha
    item_weights = []
    for _ in range(N):
      weight = int(file.readline().strip())
      item_weights.append(weight)
  
  return N, C, item_weights

In [78]:
def write_output_to_file(file_path, best_fitness):
  with open(file_path, 'w') as file:
    file.write(f'Numero de bins usados (fitness): {best_fitness}\n')

In [79]:
input_file = 'input.in'
output_file = 'output.out'

n, capacidade_bin, pesos_objetos = read_input_from_file(input_file)

In [80]:
# Função para atribuir IDs aos objetos
def atribuir_ids(objetos_pesos):
    objetos = []
    for idx, peso in enumerate(objetos_pesos, start=1):
        objeto = {'id': idx, 'peso': peso}
        objetos.append(objeto)
    return objetos

In [81]:
# Atribuindo IDs aos objetos
objetos = atribuir_ids(pesos_objetos)

In [82]:
# First-Fit
def heuristica_first_fit(objetos, capacidade_bin):
  bins = []
  for obj in objetos:
    colocado = False
    for bin in bins:
      if sum(item['peso'] for item in bin) + obj['peso'] <= capacidade_bin:
        bin.append(obj)
        colocado = True
        break
    if not colocado:
      bins.append([obj])
  return bins

In [83]:
def gerar_individuo(objetos, capacidade_bin):
    objetos_aleatorios = objetos[:]
    random.shuffle(objetos_aleatorios)
    individuo = heuristica_first_fit(objetos_aleatorios, capacidade_bin)
    return individuo

In [84]:
def crossover(pai1, pai2, capacidade_bin):
    # Fazendo cópias dos pais
    filho = [bin.copy() for bin in pai1]
    bins_pai2 = [bin.copy() for bin in pai2]

    # Escolho pontos aleatorios de cruzamento
    ponto_corte_pai1_inicio = random.randint(0, len(filho)-1)
    ponto_corte_pai1_fim = random.randint(ponto_corte_pai1_inicio, len(filho))
    ponto_corte_pai2_inicio = random.randint(0, len(bins_pai2)-1)
    ponto_corte_pai2_fim = random.randint(ponto_corte_pai2_inicio, len(bins_pai2))

    # Extrai as bins dos pais
    bins_injetadas = bins_pai2[ponto_corte_pai2_inicio:ponto_corte_pai2_fim]

    # Insiro as bins no filho
    filho = filho[:ponto_corte_pai1_inicio] + bins_injetadas + filho[ponto_corte_pai1_fim:]

    # guardo todos os ids apos a inserção
    ids_objetos_filho = [obj['id'] for bin in filho for obj in bin]

    # Identifico os ids de objetos duplicados
    ids_duplicados = set([id_obj for id_obj in ids_objetos_filho if ids_objetos_filho.count(id_obj) > 1])

    # Removendo bins que contêm objetos duplicados
    filho_sem_duplicatas = []
    for bin in filho:
        if not any(obj['id'] in ids_duplicados for obj in bin):
            filho_sem_duplicatas.append(bin)
        else:
            # Remover os ids duplicados da lista geral
            ids_duplicados -= set(obj['id'] for obj in bin)

    # Identifico ids de objetos faltantes
    ids_todos_objetos = set(obj['id'] for obj in objetos)
    ids_presentes = set(obj['id'] for bin in filho_sem_duplicatas for obj in bin)
    ids_faltantes = ids_todos_objetos - ids_presentes

    # Reinsiro os itens usando First-Fit-Decreasing
    if ids_faltantes:
        # Obter os objetos faltantes a partir dos ids
        objetos_faltantes = [obj for obj in objetos if obj['id'] in ids_faltantes]
        # Da reverse
        ob = sorted(objetos_faltantes, key=lambda obj: obj['peso'], reverse=True)
        # Reinsere os elementos usando FFD
        bins_reinseridos = heuristica_first_fit(ob, capacidade_bin)
        filho_final = filho_sem_duplicatas + bins_reinseridos
    else:
        filho_final = filho_sem_duplicatas

    return filho_final

In [85]:
# Função de mutação
def mutacao(individuo, capacidade_bin):

    individuo_mutado = [bin.copy() for bin in individuo]

    # Garantindo que sempre eliminaremos pelo menos três bins(regra de implementação no artigo)
    num_bins = len(individuo_mutado)
    num_eliminar = max(3, num_bins // 2) 

    # Identifico a bin mais vazia(ela tem que ser deletada)
    bins_ordenadas_por_vazio = sorted(individuo_mutado, key=lambda bin: sum(obj['peso'] for obj in bin))
    bin_mais_vazia = bins_ordenadas_por_vazio[0]

    # adiciono a bin mais vazia para retirar
    bins_para_eliminar = [bin_mais_vazia]

    # adiciono as outras para serem eliminadas aleatoriamente
    bins_restantes = [bin for bin in individuo_mutado if bin != bin_mais_vazia]
    bins_selecionadas_aleatorias = random.sample(bins_restantes, min(num_eliminar - 1, len(bins_restantes)))
    bins_para_eliminar.extend(bins_selecionadas_aleatorias)

    # Removo as bins
    for bin in bins_para_eliminar:
        individuo_mutado.remove(bin)

    # Coletando os objetos das bins eliminadas
    objetos_para_reinserir = [obj for bin in bins_para_eliminar for obj in bin]
    random.shuffle(objetos_para_reinserir)  # Embaralhar os objetos para diversidade

    # Reinserindo os objetos usando heurística First-Fit
    for obj in objetos_para_reinserir:
        colocado = False
        for bin in individuo_mutado:
            if sum(item['peso'] for item in bin) + obj['peso'] <= capacidade_bin:
                bin.append(obj)
                colocado = True
                break
        if not colocado:
            # Se não couber em nenhuma bin existente, criar uma nova bin
            individuo_mutado.append([obj])

    return individuo_mutado

In [86]:
def inversao(individuo):
    individuo_invertido = individuo.copy()

    # Seleciono pontos aleatorios para swapar(troca de bins), apos swapar retorna o individuo
    # essa troca de acordo com o artigo aumenta a chance de aproximar bins otimas, ou seja, o
    # swap pode aproximar bins que sobram o minimo de espaço possivel e, isso aumenta a chance 
    # de passar essa qualidade pra proxima geração
    
    ponto_inicio = random.randint(0, len(individuo_invertido) - 2)
    ponto_fim = random.randint(ponto_inicio + 1, len(individuo_invertido) - 1)
    sublista_bins = individuo_invertido[ponto_inicio:ponto_fim + 1]
    sublista_bins_invertida = sublista_bins[::-1]
    individuo_invertido[ponto_inicio:ponto_fim + 1] = sublista_bins_invertida

    return individuo_invertido

In [87]:
num_geracoes = 20
tamanho_populacao = 10
prob_crossover = 0.9
prob_mutacao = 0.7
prob_inversao = 0.9

def fitness(individuo):
    return len(individuo) # se o numero de bins do individuo é o minimo possivel significa que ele aproveitou ao maximo a distribuição dos itens

def init_pop(tamanho_populacao, objetos, capacidade_bin):
    populacao = []
    for _ in range(tamanho_populacao):
        individuo = gerar_individuo(objetos, capacidade_bin)
        populacao.append(individuo)
    return populacao

# Função de Seleção por Torneio
def selecao_torneio(populacao, tamanho_torneio=3):
    melhor = None
    for _ in range(tamanho_torneio):
        individuo = random.choice(populacao)
        if (melhor is None) or (fitness(individuo) < fitness(melhor)):
            melhor = individuo
    return melhor

populacao = init_pop(tamanho_populacao, objetos, capacidade_bin)

global_solution = gerar_individuo(objetos, capacidade_bin)

for geracao in range(num_geracoes):

    nova_populacao = []

    aptidoes = [fitness(individuo) for individuo in populacao]
    # Coletar dados para plotagem
    melhor_aptidao = min(aptidoes)
    
    while len(nova_populacao) < tamanho_populacao:
        pai1 = selecao_torneio(populacao)
        pai2 = selecao_torneio(populacao)
        
        if random.random() < prob_crossover:
            filho = crossover(pai1, pai2, capacidade_bin)
        else:
            filho = pai1.copy()
        
        if random.random() < prob_mutacao:
            filho = mutacao(filho, capacidade_bin)
        
        if random.random() < prob_inversao:
            filho = inversao(filho)
        
        nova_populacao.append(filho)
    
    populacao = nova_populacao
    
    melhor_individuo = min(populacao, key=fitness)
    
    if(fitness(global_solution) > fitness(melhor_individuo)):
        global_solution = melhor_individuo
    
    print(f"Geração {geracao + 1}: Melhor Aptidão = {fitness(melhor_individuo)}")

write_output_to_file(output_file, fitness(global_solution))
print(f"global_solution {fitness(global_solution)}")

#1000 411  16.4s
#500 210   4.4s
#5245 

Geração 1: Melhor Aptidão = 2181
Geração 2: Melhor Aptidão = 2155
Geração 3: Melhor Aptidão = 2155
Geração 4: Melhor Aptidão = 2150
Geração 5: Melhor Aptidão = 2150
Geração 6: Melhor Aptidão = 2135
Geração 7: Melhor Aptidão = 2135
Geração 8: Melhor Aptidão = 2135
Geração 9: Melhor Aptidão = 2160
Geração 10: Melhor Aptidão = 2158
Geração 11: Melhor Aptidão = 2149
Geração 12: Melhor Aptidão = 2148
Geração 13: Melhor Aptidão = 2145
Geração 14: Melhor Aptidão = 2144
Geração 15: Melhor Aptidão = 2135
Geração 16: Melhor Aptidão = 2135
Geração 17: Melhor Aptidão = 2147
Geração 18: Melhor Aptidão = 2134
Geração 19: Melhor Aptidão = 2157
Geração 20: Melhor Aptidão = 2151
global_solution 2134
